# Part 3 - texto

- maior/menor louvor
- word length (maybe)
- bag-of-words with frequency
- gensim corpus
- tf-idf
- named entity recognition (spacy or polyglot)
- next: category classification for avulsos

## Maior e menor louvor

In [ ]:
import pandas as pd
from pathlib import Path

assets_folder = Path("../assets")
hinos_analise = pd.read_pickle(assets_folder / "hinos_analise.pkl")
hinos_analise = hinos_analise.set_index("numero")
hinos_analise["categoria_abr"] = hinos_analise["categoria"].apply(
    lambda x: x[:13] + "..." if len(x) > 15 else x
)
hinos_analise

In [ ]:
import nltk
from tqdm import tqdm

nltk.download("stopwords")

stopwords = nltk.corpus.stopwords.words("portuguese")
stopwords.extend(["ó", "ti", "pra", "lo", "oh"])
text = []
text_no_stops = []

for hino in tqdm(hinos_analise.to_dict("records")):
    tokens = nltk.tokenize.regexp_tokenize(hino["texto_limpo"], r"\w+")
    # Replace "MINH" with "MINHA" with regex
    tokens = [nltk.re.sub(r"^MINH$", "MINHA", palavra) for palavra in tokens]
    tokens = [
        palavra for palavra in tokens if palavra != "CORO" and palavra != "INSTRUMENTOS"
    ]
    tokens_no_stops = [
        palavra for palavra in tokens if palavra.lower() not in stopwords
    ]
    text.append(tokens)
    text_no_stops.append(tokens_no_stops)

hinos_analise["tokens"] = text
hinos_analise["tokens_no_stops"] = text_no_stops
# considerando numero total de palavras, pois todas elas tem que ser cantadas, logo impactam no tamanho prático do hino
hinos_analise["num_tokens"] = hinos_analise["tokens"].apply(len)
hinos_analise

In [ ]:
display(hinos_analise.sort_values("num_tokens", ascending=False))
display(hinos_analise.sort_values("num_tokens", ascending=True))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure 'categoria_id' is treated as a categorical variable
hinos_analise["categoria_id"] = hinos_analise["categoria_id"].astype("category")

# Create a mapping between categoria_id and categoria
categoria_mapping = (
    hinos_analise[["categoria_id", "categoria_abr"]]
    .drop_duplicates()
    .set_index("categoria_id")["categoria_abr"]
)

# Create a violin plot
plt.figure(figsize=(12, 6))
""" sns.violinplot(
    data=hinos_analise,
    x="categoria_id",
    y="num_tokens",
    palette="viridis",
    inner="quartile",
) """
sns.boxplot(data=hinos_analise, x="categoria_id", y="num_tokens", palette="viridis")

# Replace x-ticks with corresponding 'categoria' names
plt.xticks(
    ticks=range(len(categoria_mapping)),
    labels=categoria_mapping,
    rotation=90,
    ha="right",
)

# Add labels and title
plt.xlabel("Categoria")
plt.ylabel("Number of Tokens")
plt.title("Relationship Between Number of Tokens and Categoria (Violin Plot)")

# Show the plot
plt.tight_layout()
plt.show()

## Word length (maybe)

- bag-of-words with frequency + word map
- gensim corpus
- tf-idf
- named entity recognition (spacy or polyglot)

In [ ]:
tokenized_lines = hinos_analise.iloc[0]["tokens_no_stops"]
tokenized_lines[:10]

In [ ]:
# Make a frequency list of lengths: line_num_words
line_num_words = [len(t_line) for t_line in hinos_analise["tokens_no_stops"].explode().tolist()]

# Plot a histogram of the line lengths
plt.hist(line_num_words)

# Show the plot
plt.show()

## Palavras

In [ ]:
palavras = hinos_analise["tokens_no_stops"].explode().tolist()

### Palavras mais longas

In [ ]:
# find the 10 largest words
palavras_unique = list(set(palavras))
palavras_unique.sort(key=len, reverse=True)
print(len(palavras_unique))
pd.DataFrame({
    "palavra": palavras_unique[:10],
    "tamanho": [len(palavra) for palavra in palavras_unique[:10]]
})

### Bag-of-words with frequency

In [ ]:
print(len(palavras))
set_words_full = list(set(palavras))
count_words = [palavras.count(i) for i in set_words_full]

contagem_palav = pd.DataFrame(
    zip(set_words_full, count_words), columns=["palavra", "contagem"]
)
contagem_palav = contagem_palav.sort_values("contagem", ascending=False)
contagem_palav

In [ ]:
# create a percentage column
contagem_palav["percentual"] = contagem_palav["contagem"] / len(palavras) * 100
contagem_palav

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create a dictionary from the word frequency data
word_freq_dict = dict(zip(contagem_palav['palavra'], contagem_palav['contagem']))

# Generate word cloud
wordcloud = WordCloud(
    width=800, 
    height=400, 
    background_color='white',
    max_words=100,
    colormap='viridis',
    relative_scaling=0.5,
    random_state=42
).generate_from_frequencies(word_freq_dict)

# Plot the word cloud
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud - Palavras mais frequentes nos hinos', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# Also show top 20 most frequent words as a bar chart
plt.figure(figsize=(12, 8))
top_20 = contagem_palav.head(20)
plt.barh(range(len(top_20)), top_20['contagem'], color='skyblue')
plt.yticks(range(len(top_20)), top_20['palavra'])
plt.xlabel('Frequência')
plt.title('Top 20 Palavras Mais Frequentes')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Named Entity Recognition

Tentei:
- Gensim corpus (problemas de compatibilidade)
- NLTK NER (ruim)
- Polyglot (não consegui instalar)
- SpaCy (péssimos resultados)

In [ ]:
texto_completo = " ".join(hinos_analise["texto_limpo"])
texto_completo[:100]

In [ ]:
import spacy

# python -m spacy download pt_core_news_lg
nlp = spacy.load("pt_core_news_lg")
doc = nlp(texto_completo)
doc.ents[:5]

In [ ]:
for ent in doc.ents[:25]:
    print(ent.text, ent.label_)
